In [70]:
"""
Arjun Srivastava
arj1
AMATH 301 B
"""

import numpy as np
import scipy.linalg
import time

In [67]:
# Problem 1

# a)

a, b = 1e21 - (1e21 - 1e5), (1e21 - 1e21) + 1e5

diff = abs(a-b)  # diff = 31072.0

"""
Considering the size of the numbers a and b, I would initially say that this result is not very significant. 31072.0, however,
is still a large number, and there are many contexts in which this error would be significant.
"""

# b)

A = np.array([[10e-20, 1], [1, 1]])

cond = np.linalg.cond(A)  # cond = 2.6180339887498953, which implies a unique solution

# c)

L, U = np.array([[1, 0], [10e20, 1]]), np.array([[10e-20, 1], [0, 1-10e20]])
print('='*30, 'c)', sep='\n')
print(A, (L @ U), sep='\n\n')

"""
L @ U is not equal to A. The first row is, and this is because we are not adding values that involve catastrophic cancellation.
The bottom row, however, involves 10e20 + 10e-20, which should be equal to 0. As can be seen by the result printed below,
there is an extra 100. This makes sense due to the immense size of 10e20 and 10e-20. The second column of the second row
has 10e20 - 10e20. The result should be 1, but it is instead 0. Catastrophic cancellation is occurring in both of these cases
since 10e20 and 10e-20 are being joined by arithmetic operations.
"""

# d)

B = np.array([[1, 1], [10e-20, 1]])
L1, U1 = np.array([[1, 0], [10e-20, 1]]), np.array([[1, 1], [0, 1-10e-20]])
print('='*30, 'd)', sep='\n')
print(B, (L1 @ U1), sep='\n\n')
print('='*30, 'e)', sep='\n')

"""
By reordering the matrix A to B, we have avoided any operation that involves two very large numbers (10e20, 10e-20). Since
10e20 and 10e-20 are only ever being added to or subtracted by 1, there is no case in which catastrophic cancellation occurs,
and our answers are much more accurate.
"""

# e)

P, L2, U2 = scipy.linalg.lu(A)
P = P.T
print(L2, U2, P, sep='\n')
print('\n')

P1, L3, U3 = scipy.linalg.lu(B)
P1 = P1.T
print(L3, U3, P1, sep='\n')

"""
The official SciPy LU yields the same results as part d). This implies that Python is using the permutation matrix
to ensure that the matrix A can be more efficiently decomposed without issues. As shown, P is different for A and B,
implying that Python wants to avoid catastrophic cancellation by setting up the equation in a more efficient manner. Since
A is not set up efficiently for an LU decomp, the permutation matrix converts it.
"""

c)
[[1.e-19 1.e+00]
 [1.e+00 1.e+00]]

[[1.e-19 1.e+00]
 [1.e+02 0.e+00]]
d)
[[1.e+00 1.e+00]
 [1.e-19 1.e+00]]

[[1.e+00 1.e+00]
 [1.e-19 1.e+00]]
e)
[[1.e+00 0.e+00]
 [1.e-19 1.e+00]]
[[1. 1.]
 [0. 1.]]
[[0. 1.]
 [1. 0.]]


[[1.e+00 0.e+00]
 [1.e-19 1.e+00]]
[[1. 1.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]]


'\nThe official SciPy LU yields the same results as part d). This implies that Python is using the permutation matrix\nto ensure that the matrix A can be more efficiently decomposed without issues. If P we\n'

In [68]:
# Problem 2

# a)

A = np.genfromtxt('example_matrix.csv', delimiter=',')

In [ ]:
# b)

t0 = time.time()
for i in range(100):
    b = np.random.rand(3000, 1)
    x = scipy.linalg.solve(A, b)
t1 = time.time()
    